# Running an airspace simulation

This notebook is intended to run through uploading an "I"-shaped sector into a simulation and the basics of interacting with `BlueSky` via `pydodo`.
For a full overview of the `pydodo` commands, please see the [pydodo specification document](https://github.com/project-bluebird/dodo/blob/master/Specification.md).

## Contents

* [1. Getting started](#first-section)
* [2. Load sector and scenario](#second-section)
* [3. Example usage](#third-section)
* [4. Example OpenAI interface](#fourth-section)
* [5. Shut down the simulator](#fifth-section)

## 1. Getting started <a class="anchor" id="first-section"></a>

### 1.1 Launch simulator (`BlueSky`) and the `BlueBird` interface layer

If you have not already started the BlueSky simulator and BlueBird, you can launch them using either docker, by navigating to the [Simurgh](https://github.com/alan-turing-institute/simurgh.git) repository that contains the docker container and running...

In [1]:
!docker-compose --file docker-compose.yml up --detach

Creating network "guides_default" with the default driver
Creating bluesky ... 
Creating bluebird ... done
Creating twitcher ... mdone
ting twitcher ... done

or via the command line as outlined in the installation instructions page

### 1.2 Import pydodo
Pydodo will be used to communicate with the simulator

In [2]:
import pydodo

To check pydodo is comminicating with BlueBird:

In [3]:
print(pydodo.bluebird_connect.get_bluebird_url())

http://localhost:5001/api/v2


If this is not correct (e.g., BlueBird is in fact being run on a different host), then we can change the url by using the `pydodo.bluebird_config` function:
For help on pydodo type

In [4]:
help(pydodo.bluebird_config)

Help on function bluebird_config in module pydodo.bluebird_connect:

bluebird_config(host='localhost', port='5001', version='v2')
    Set BlueBird host, port and version parameters.
    Default values are taken from the config file.
    
    Parameters
    ----------
    host : str
        BlueBird host (e.g., 'localhost' or '0.0.0.0').
    port : int
        BlueBird port (e.g., 5001).
    version : str
        BlueBird version (e.g., 'v1' or 'v2')



At the moment there is no scenario or sector information loaded, however we can confirm that pydodo is able to report on the type, speed, position, headings and a range of other aircraft parameters by typing

In [5]:
pydodo.all_positions()

,aircraft_type,ground_speed,latitude,longitude,vertical_speed,heading,current_flight_level,requested_flight_level,cleared_flight_level


## 2. Load sector and scenario <a class="anchor" id="second-section"></a>

### 2.1 Overview

To start, we need to upload a **sector** and a **scenario** (in that order). 

A sector defines the area the ATCO is controling. The scenario defines the episode: that is a sequence of aircraft whose route passes through the sector.

You can use the [Aviary](https://github.com/alan-turing-institute/aviary/tree/develop) package to create sector and scenario files (see the README for more information and file formats).

We can check whether a sector or a scenario has been defined already (see whether `sector_name` and `scenario_name` below are not `None`):

In [6]:
pydodo.simulation_info()

{'dt': 0.05,
 'mode': 'Agent',
 'scenario_name': None,
 'scenario_time': 0.0,
 'sector_name': None,
 'seed': None,
 'sim_type': 'BlueSky',
 'speed': 1.0,
 'state': 'INIT',
 'utc_datetime': '2021-09-01 00:00:00',
 'aircraft_ids': []}

Nothing has been loaded yet

### 2.2 Load sector and scenario into BlueSky using BlueBird

In [7]:
pydodo.upload_sector('I-sector.geojson', 'I_sector')

True

In [8]:
pydodo.upload_scenario('test-scenario.json', 'test_scenario')

True

We can check that the sector and scenario have been read into the simulation via:

In [9]:
pydodo.simulation_info()

{'dt': 0.05,
 'mode': 'Agent',
 'scenario_name': 'test_scenario',
 'scenario_time': 1.2,
 'sector_name': 'I_sector',
 'seed': None,
 'sim_type': 'BlueSky',
 'speed': 1,
 'state': 'RUN',
 'utc_datetime': '2021-09-01 00:00:01',
 'aircraft_ids': ['VJ159', 'VJ405']}

In [10]:
pydodo.all_positions()

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ159,A346,400,39996.25,198,0,49.393529,-0.1275,400,-360
VJ405,B77W,200,20000.00,139,180,53.573276,-0.1275,400,0


## 2.3 Viewing the simulation

You can also have a look at the sector and scenario in the simulation through Twitcher by going to http://localhost:8080.

If the "I" sector was loaded the Twitcher page should look like this. ![I-sector as loaded in Twitcher](../images/twitcher_i_sector_initial.png)

## 3. Example usage <a class="anchor" id="third-section"></a>

See the Dodo [Specification document](https://github.com/alan-turing-institute/dodo/blob/master/Specification.md) for a detailed overview of the supported commands.

Below we show some of them.

### 3.1 Query aircraft positions, again

In [11]:
pydodo.all_positions()

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ159,A346,400,39996.25,198,0,49.393529,-0.1275,400,-360
VJ405,B77W,200,20000.00,139,180,53.573276,-0.1275,400,0


### 3.2 Check aircraft route

In [12]:
pydodo.list_route('VJ159')

{'next_waypoint': 'FIYRE',
 'route_name': 'ASCENSION',
 'route_waypoints': ['FIYRE', 'EARTH', 'WATER', 'AIR', 'SPIRT'],
 'aircraft_id': 'VJ159'}

In [13]:
pydodo.list_route('VJ405')

{'next_waypoint': 'SPIRT',
 'route_name': 'FALLEN',
 'route_waypoints': ['SPIRT', 'AIR', 'WATER', 'EARTH', 'FIYRE'],
 'aircraft_id': 'VJ405'}

### 3.3 Step through the simulation

By default, the simulation is in a paused state. To advance it, we need to call the `simulation_step` function:

In [14]:
for i in range(0,50):
    pydodo.simulation_step()

This should result in the aircraft positions changing:

In [15]:
pydodo.all_positions()

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ159,A346,400,38794.5,215,0,49.486888,-0.1275,400,-1500
VJ405,B77W,200,20000.0,164,180,53.505683,-0.1275,400,0


### 3.4 Change altitude

We can also instruct the aircraft to change their path (e.g., heading or altitude).

Below we instruct aircraft VJ405 to start climbing. This is followed by a step through the simulation:

In [16]:
pydodo.change_altitude("VJ405", flight_level = 400)

True

In [17]:
pydodo.simulation_step()

True

Checking aircraft positions now, we should see a change in the `current_flight_level` as the aircraft starts climbing: Note also the cleared flight level has increased. There may be some uncertainty in this measurement as to weather the cleared flight level is now reported in feet or as a flight level. It is something that as of Summer 2021 is being looked into

In [18]:
pydodo.aircraft_position("VJ405")

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ405,B77W,40000,20000.0,164,180,53.504201,-0.1275,400,0


## 4. Example OpenAI interface <a class="anchor" id="fourth-section"></a>
This is still to do, another feature we are looking at is the logging feature which should be available for agent mode I think

## 5. Shut down the simulator <a class="anchor" id="fifth-section"></a>

In [19]:
!docker-compose down

Stopping twitcher ... 
Stopping bluebird ... 
Stopping bluesky  ... 
Removing twitcher ... mdone
Removing bluebird ... 
Removing bluesky  ... 
Removing network guides_default
